# #)1112번,9번,5100번 버스도 똑같은 수집 코드를 이용합니다.

# 1) 필요한 클래스 및 함수 정의

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
import time
import sklearn.preprocessing

pd.set_option("display.max_columns", None)
warnings.filterwarnings(action='ignore')

serviceKey='bwUUK6MEA6GnqDpo2ZS9ChN4VZ1nkUPXrryU80XxUcLqxB62pIGIPWXNdyzqgT20BHrw8zxWWk9rqiSQwTrbnQ=='
p5100 ={'serviceKey' :serviceKey,
             'routeId':'200000115','stationId':'228000703'}# stationid:외대
p1112 ={'serviceKey' : serviceKey,
         'routeId':'234000016','stationId':'228000703'}
p0009 ={'serviceKey' :serviceKey,
             'routeId':'200000103','stationId':'228000703'}
p7000 ={'serviceKey' :serviceKey,
             'routeId':'200000112','stationId':'228000703'}

In [3]:
class Bus():
    def __init__(self,params):
        self.params=params
        self.bus_time=[]
        
    def repet(self):
        url = 'http://apis.data.go.kr/6410000/busarrivalservice/getBusArrivalItem'
        response = requests.get(url, params=self.params)
        soup=BeautifulSoup(response.text,'xml')
        return soup

        
    def current_buses(self): 
        url = 'http://apis.data.go.kr/6410000/buslocationservice/getBusLocationList'
        mod_params ={'serviceKey' : self.params['serviceKey'],
                  'routeId':self.params['routeId']}

        response=requests.get(url,params=mod_params)
        soup=BeautifulSoup(response.text,'xml')
        locations=sorted([int(x.stationSeq.text) for x in soup.find_all('busLocationList')])
        return locations


def trim_data(df,FileName):
    df['시간'] = df['시간'].apply(pd.to_datetime)
    df['hour']=df['시간'].dt.hour
    df['minute']=df['시간'].dt.minute
    df['day'] = df['시간'].dt.weekday 

    mb = sklearn.preprocessing.MultiLabelBinarizer()
    if isinstance(df['운행중인 다른 버스들의 위치'].iloc[0], str):
        df['운행중인 다른 버스들의 위치'] = df['운행중인 다른 버스들의 위치'].apply(eval)
        
    entity = mb.fit_transform(df['운행중인 다른 버스들의 위치']) 
    lsts=pd.DataFrame(entity,columns=mb.classes_)
    lsts.columns=[f'station{x}' for x in lsts.columns]
    df=pd.concat([df,lsts],axis=1,sort=False).reset_index(drop=True)


    df=df.drop(["운행중인 다른 버스들의 위치"],axis=1)
    
    batch=df.index[df.arrived==1]
    df=df.iloc[:batch[-1]+1,:]

    batch=df.index[df.arrived==1]
    temp=0
    for x in batch:
        for y in range(temp,x): 
            df['arrived'][y]=(df['시간'][x]-df['시간'][y]).seconds
            temp=x+1
    for e in batch:
        df['arrived'][e]=0

    df = df.set_index('시간')
    df.to_csv(f'./사색역데이터/{FileName}')   
    return df

# 2)데이터 수집코드

In [10]:
past=4
pBus=Bus(p7000)
for x in range(340): # 12시간
    
    try:
        soup=pBus.repet()
        result=int(soup.resultCode.text)
        arrived=0
        if past!=result:
            if result==0:
                arrived=1
        pBus.bus_time.append([soup.queryTime.text,pBus.current_buses(),arrived])
        past=result
        print(x,"번째 시도: ",pBus.bus_time)
        time.sleep(60)
        
    except requests.Timeout as err:
        continue
    except Exception as e:
        continue

df=pd.DataFrame(pBus.bus_time,columns=['시간','운행중인 다른 버스들의 위치','arrived'])

mod_df=trim_data(df,'7000820.csv')
mod_df

0 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0]]
1 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0]]
2 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0]]
3 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0]]
4 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 

16 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439',

22 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439',

27 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439',

31 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439',

...

336 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439'

337 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439'

338 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439'

339 번째 시도:  [['2022-08-20 16:52:18.445', [10, 16, 27, 31, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:53:18.648', [10, 16, 27, 32, 40, 41, 53, 57, 65, 79], 0], ['2022-08-20 16:54:18.773', [11, 16, 27, 32, 41, 41, 54, 58, 65, 79], 0], ['2022-08-20 16:55:19.053', [11, 17, 28, 33, 41, 42, 54, 59, 68, 79], 0], ['2022-08-20 16:56:22.803', [11, 18, 28, 33, 41, 42, 55, 60, 68, 79], 0], ['2022-08-20 16:57:23.661', [12, 18, 29, 33, 42, 43, 55, 60, 68, 79], 0], ['2022-08-20 16:58:23.817', [12, 18, 29, 33, 42, 44, 55, 61, 68, 79], 0], ['2022-08-20 16:59:24.035', [13, 19, 30, 34, 43, 44, 55, 61, 68, 79], 0], ['2022-08-20 17:00:24.192', [2, 13, 19, 31, 34, 44, 45, 56, 62, 69, 79], 1], ['2022-08-20 17:01:24.426', [3, 14, 19, 31, 36, 44, 45, 56, 63, 70], 0], ['2022-08-20 17:02:24.597', [4, 14, 20, 31, 36, 45, 46, 56, 63, 70], 0], ['2022-08-20 17:03:24.831', [5, 15, 20, 32, 37, 45, 46, 57, 64, 70], 0], ['2022-08-20 17:04:25.111', [5, 16, 20, 33, 37, 46, 47, 57, 64, 72], 0], ['2022-08-20 17:05:25.439'

,arrived,hour,minute,day,station1,station2,station3,station4,station5,station6,station7,station8,station9,station10,station11,station12,station13,station14,station15,station16,station17,station18,station19,station20,station21,station22,station23,station24,station25,station26,station27,station28,station29,station30,station31,station32,station33,station34,station35,station36,station37,station38,station39,station40,station41,station42,station43,station44,station45,station46,station47,station48,station49,station50,station51,station52,station53,station54,station55,station56,station57,station58,station59,station60,station61,station62,station63,station64,station65,station66,station67,station68,station69,station70,station71,station72,station73,station74,station75,station76,station77,station78,station79
시간,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-08-20 16:52:18.445,485,16,52,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2022-08-20 16:53:18.648,425,16,53,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2022-08-20 16:54:18.773,365,16,54,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2022-08-20 16:55:19.053,305,16,55,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2022-08-20 16:56:22.803,241,16,56,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-20 22:16:37.664,240,22,16,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2022-08-20 22:17:37.866,180,22,17,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2022-08-20 22:18:38.068,120,22,18,5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


# 3) 최종 데이터

In [6]:
data7000=pd.read_csv('./사색역데이터/7000_combined.csv')
data7000

,시간,arrived,hour,minute,day,station1,station2,station3,station4,station5,station6,station7,station8,station9,station10,station11,station12,station13,station14,station15,station16,station17,station18,station19,station20,station21,station22,station23,station24,station25,station26,station27,station28,station29,station30,station31,station32,station33,station34,station35,station36,station37,station38,station39,station40,station41,station42,station43,station44,station45,station46,station47,station48,station49,station50,station51,station52,station53,station54,station55,station56,station57,station58,station59,station60,station61,station62,station63,station64,station65,station66,station67,station68,station69,station70,station71,station72,station73,station74,station75,station76,station77,station78,station79
0,2022-08-15 16:44:03.616,448,16,44,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2022-08-15 16:45:04.840,387,16,45,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2022-08-15 16:46:08.034,324,16,46,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2022-08-15 16:47:08.196,264,16,47,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2022-08-15 16:48:11.404,200,16,48,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,2022-08-20 22:16:37.664,240,22,16,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1235,2022-08-20 22:17:37.866,180,22,17,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1236,2022-08-20 22:18:38.068,120,22,18,5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1237,2022-08-20 22:19:38.270,60,22,19,5,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


# 4)데이터 feature 설명

### - Index= 현 시각
### - Arrived= 몇 초 뒤에 사색의 광장역에 버스가 도착하는지. 따라서 arrived=0인 열은 그 시각에 버스가 사색역에 도착했다는 뜻입니다.
### - Hour,minute= 현 시간
### - Station1~station79= 현 시각 운행중인 모든 1112번 버스들의 위치. 예를 들어, 한 1112번 버스가 station4에 있다면 station4=1로 표현됩니다. station10에는 아무 1112번 버스도 없다면 0으로 표시됨.
